# Malicious scenario

First, Bob will send his data point (without label) to Alice. Alice runs inference on her model and sends the hash of the prediction, with a ZKP, back to Bob. Bob verifies the ZKP, then participates in the MPC protocol to get the loss of the model.

In [2]:
from torch import nn
import ezkl
import os
import json
import torch

In [3]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        # reduce to single value
        logits = torch.mean(logits, dim=1)

        return logits


circuit = MyModel()


shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
# x = 0.1*torch.rand(1,*shape, requires_grad=True)
x = torch.ones(1,*shape, requires_grad=True) * 3
print(x.shape)

torch.Size([1, 1, 28, 28])


In [4]:
#Specifying some path parameters
model_path = os.path.join('data','network.onnx')
compiled_model_path = os.path.join('data','network.compiled')
pk_path = os.path.join('data','test.pk')
vk_path = os.path.join('data','test.vk')
settings_path = os.path.join('data','settings.json')

witness_path = os.path.join('data','witness.json')
data_path = os.path.join('data','input.json')
output_path = os.path.join('data','output.json')

In [5]:
# Flips the neural net into inference mode
circuit.eval()

out = circuit(x)
print(out)


    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))

out_array = out.detach().numpy().tolist()
output = dict(output_data = out_array)

    # Serialize data into file:
json.dump( output, open(output_path, 'w' ))

tensor([0.0303], grad_fn=<MeanBackward1>)


In [6]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public" #Bob can see this
py_run_args.output_visibility = "hashed/public" #This hash is given to Bob
py_run_args.param_visibility = "private" 

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)
assert res == True


In [7]:
cal_path = os.path.join('data',"calibration.json")

#Alice should use some real data to calibrate the model, here we use random data
data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True
# srs path
res = await ezkl.get_srs( settings_path)
res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

[tensor] decomposition error: integer -317645693 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -1281068661 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -5127811340 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -2563294932 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -10260280118 is too large to be represented by base 16384 and n 2
forward pass failed: "failed to forward: [halo2] General synthesis error"
[tensor] decomposition error: integer -30261717452 is too large to be represented by base 16384 and n 2
forward pas

In [8]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [9]:
# GENERATE A PROOF


proof_path = os.path.join('data','test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [['...']], 'proof': '0x009e4883de847054c7a732bae99390cb4156848252b000815e2b7a8300fa8b6a22b868def6df413b3e86de6c1adf83b842ccf2970eacea2d8075e792b5e2302d16c055d14fbbc47d136afae050b417cb5b084205cf38d08bd16502cd63cf38192794e2e7a61f24c77dc570c995a352763e0b9739249cdc29d9e7e2f4d1cb04142345bed1a2bd0d82435743c9cb912965ef6466c8ca5310054569adc2e703021b2f659dc0e3ea1bc07fadcf3a0714f108945a2be0283ee63bfbd7a4a0d6875769173951dbdc4c120436927d3b4f7403757fdd932dd19d7afea3e6987690cbea5f2dcd28fb880e4634fa033215e24fa02cc81bf9848d6050b5902a44d9aef44566154afbf000299d2cb9653b4d3a72c9ed8ba027c0acb58c1941b6e0f1ad1add843038393049d230f47bc4965730279ecfe1c6f2748b41a5b7bb32e5b3a271165215dccd2e6786cc06f137a7a13c0cf0f7de098807dbf6794835cdb633329e540b176f442721e96df77e6c44ff4845f4c1a0bdbf5101ee7dbe69e0cde868e2329912b1ada8c701c60a5361d196bc665c6ad0664ecd9a2fba6160ea44b22f53c343188cbc760c2ee1d4410d87f554f643c996d4830c2a03e1d7b3414f94b9b9545f095ff4b42d59e9ea67b8419fb6435cce7060d1fb9ba7f4406565c8394d7c975b219

In [10]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

verified
